In [58]:
import sys, os, time, json, gc

from argparse import Namespace
from utils.args import init_args
from utils.hyperparams import hyperparam_path
from utils.initialization import *
from utils.example import Example
from utils.batch import Batch
from utils.optimization import set_optimizer
from model.model_utils import Registrable
from model.model_constructor import *

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.distributed import DistributedSampler

In [6]:
# start_time = time.time()
# if args.read_model_path and args.read_model_path != '.':
#     params = json.load(open(os.path.join(args.read_model_path, 'params.json')), object_hook=lambda d: Namespace(**d))
#     params.lazy_load = True
# elif args.read_model_path == '.':
#     params = args
# set up the grammar, transition system, evaluator, etc.


In [9]:
from functools import partial

In [7]:
Example.configuration(plm='google/electra-large-discriminator', method='lgesql')
train_dataset, dev_dataset = Example.load_dataset('train'), Example.load_dataset('dev')

# train_sampler = DistributedSampler(train_dataset)

Skip 0 extremely large samples in training dataset ...


RuntimeError: Default process group has not been initialized, please make sure to call init_process_group.

In [ ]:
train_sampler = DistributedSampler(train_dataset)

In [60]:
class collator(object):
    def __init__(self, device, train, smoothing = 0):
        self.device = device
        self.train = train
        self.smoothing = smoothing
     
    def __call__(self, batch):
        if self.smoothing:
            batched = Batch.from_example_list(batch, self.device, train=self.train, smoothing=self.smoothing)
        else:
            batched = Batch.from_example_list(batch, self.device, train=self.train)
        return batched

In [61]:
c = collator(device = 'cpu', train=False)

In [64]:
train_loader = DataLoader(train_dataset, batch_size = 20, collate_fn=c)

dev_loader = DataLoader(dev_dataset, batch_size = 20, collate_fn=c)

In [65]:
a = next(iter(dev_loader))

In [66]:
a.graph.local_g

Graph(num_nodes=786, num_edges=15364,
      ndata_schemes={}
      edata_schemes={})

In [81]:
a.graph.lg.out_degrees()

tensor([28,  9,  9,  ...,  2, 18, 17], dtype=torch.int32)

In [68]:
current_batch = Batch.from_example_list(dev_dataset[0:20], device='cpu', train=False)

In [80]:
current_batch.graph.lg.out_degrees()

tensor([28,  9,  9,  ...,  2, 18, 17], dtype=torch.int32)